In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\kripa\\Desktop\\project\\ML_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\kripa\\Desktop\\project\\ML_project'

# step 1: Updating the config.yaml 
#### config.yaml
 artifacts_root: artifacts 


data_ingestion:
  root_dir: artifacts/data_ingestion
  source_URL: https://github.com/entbappy/Branching-tutorial/raw/master/winequality-data.zip
  local_data_file: artifacts/data_ingestion/data.zip
  unzip_dir: artifacts/data_ingestion
  
 


# step 2: Updating the entity
#### src/entity/config_entity.py

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


# step 3: Updating the Configuration Manager

#### src/config

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH,
            schema_filepath= SCHEMA_FILE_PATH):

            self.config= read_yaml(config_filepath)
            self.params= read_yaml(params_filepath)
            self.schema= read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
        
         

# Step 4: Udating the Components
#### src/components/data_ingestion.py
 

In [9]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size
from src.mlProject.entity.config_entity import DataIngestionConfig


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)







            

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-12 03:12:32,918: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-12 03:12:32,918: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-12 03:12:32,918: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-12 03:12:32,926: INFO: common: created directory at: artifacts]
[2023-10-12 03:12:32,926: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-12 03:12:34,351: INFO: 166201425: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8520:2BE1BA:12C378:1D76C4:652716C7
Accept-Ranges: bytes
Date: Wed, 11 

# Step 5: Updating the pipeline
#### src/pipeline/s1_data_ingestion.py

from mlProject.config.configuration import ConfigurationManager
from mlProject.components.Data_ingestion import DataIngestion
from mlProject import logger



STAGE_NAME = "Data Ingestion stage"

class DataIngestionTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        config = ConfigurationManager()
        data_ingestion_config = config.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)
        data_ingestion.download_file()
        data_ingestion.extract_zip_file()


    
if __name__ == '__main__':
    try:
        logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
        obj = DataIngestionTrainingPipeline()
        obj.main()
        logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
    except Exception as e:
        logger.exception(e)
        raise e



# Step 6: Updating the main.py
#### from mlProject import logger
from mlProject.pipeline.stage_01_data_ingestion import DataIngestionTrainingPipeline

STAGE_NAME = "Data Ingestion stage"
try:
   logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<") 
   data_ingestion = DataIngestionTrainingPipeline()
   data_ingestion.main()
   logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
except Exception as e:
        logger.exception(e)
        raise e
